In [ ]:
import os
import cv2
import pandas as pd
import numpy as np

import torch
import matplotlib.pyplot as plt

np.set_printoptions(threshold=np.inf, linewidth=np.inf)

In [ ]:
# RLE 인코딩 함수
def rle_decode(mask_rle, shape):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)

In [ ]:
class score:
    def __init__(self,  answer_csv_file, result_csv_file, idx):
        self.answer = pd.read_csv(answer_csv_file)
        self.result = pd.read_csv(result_csv_file)
        self.idx = idx
        self.count = 0

    def img_csv(self):
        img_path = self.answer.iloc[self.idx, 1]
        image = cv2.imread(img_path)

        result = self.result.iloc[self.idx, 1]
        result_mask = rle_decode(result, (image.shape[0], image.shape[1]))

        answer = self.answer.iloc[self.idx, 2]
        answer_mask = rle_decode(answer, (image.shape[0], image.shape[1]))

        flattened_result_mask = result_mask.flatten()
        flattened_answer_mask = answer_mask.flatten()

        for i in range(len(image)*len(image)):
            if(flattened_result_mask[i] == flattened_answer_mask[i]):
                self.count+=1
            

In [ ]:
total_count=0
answer='./test.csv'
result='./submit_1024.csv'
size=714
for i in range(0, size):
    dataset = score(answer, result, i)
    dataset.img_csv()
    total_count+=dataset.count

image_len = pd.read_csv(answer)
img_path = image_len.iloc[0, 1]
image = cv2.imread(img_path)
total_score = total_count / ((len(image)*len(image))*size)    